In [1]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import ChatPromptTemplate
import base64
from PIL import Image

# LOAD GOOGLE API

In [2]:
load_dotenv()
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

# GEMINI LLM MODEL

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# PROMT

In [4]:
movie_title_promt = PromptTemplate.from_template("Suggest me only one title of a movie to watch in language {language} and of genre {genre}")

In [5]:
movie_summary_promt = PromptTemplate.from_template('Give me 2-3 line summary of the movie {movie_name}')

In [6]:
promt = movie_title_promt.invoke({
    "language": "Hindi",
    "genre": "Comedy"
})

In [7]:
result = llm.invoke(promt)
print(result.content)

**Hera Pheri**


# Langchain Chains

In [8]:
# movie_chain = movie_title_promt | llm | StrOutputParser()

In [9]:
# movie_chain = {'movie_name': movie_title_promt} | movie_summary_promt | llm | StrOutputParser()

In [10]:
movie_chain =  movie_title_promt | llm | {'movie_name': StrOutputParser()} | movie_summary_promt | llm | StrOutputParser()

In [12]:
movie_chain.invoke({'language':'Hindi','genre':'Horror'})

'"Pari" is a supernatural horror film about Arnab, who unexpectedly finds a woman named Rukhsana chained in the woods after a road accident. He takes her in, unaware that she\'s a demon haunted by an evil professor who hunts down such creatures. As Arnab tries to protect Rukhsana, he uncovers a dark and terrifying world of demonic rituals and ancient curses.'

#### This type of channing is called Sequential channing, extended channing

In [13]:
movie_chain2 = movie_title_promt | llm | StrOutputParser()
movie_title = movie_chain2.invoke({'language':'Turkish','genre':'Horror'})
print(movie_title)

movie_chain3 = movie_summary_promt | llm | StrOutputParser()
movie_summary = movie_chain3.invoke({'movie_name': movie_title})
print(movie_summary)

**Dabbe: Zehr-i Cin**
A young woman, Dilek, is plagued by increasingly terrifying supernatural events after a past traumatic experience. As she and her husband seek help, they uncover a sinister djinn curse linked to her childhood, forcing them to confront a powerful and malevolent entity.


# RunnableSequential

In [127]:
seq = RunnableSequence( movie_summary_promt, llm, StrOutputParser())

In [128]:
seq.invoke({'movie_name':movie_title})

'Dabbe (specifically Dabbe: The Possession, the first film in the series) is a Turkish horror film about a psychiatrist, Ebru, investigating the mysterious death of her childhood friend, Tarik. As she delves deeper, she uncovers a terrifying world of Djinn possession and black magic rituals, forcing her to confront her own skepticism and battle supernatural forces.'

In [174]:
command = llm.invoke('Give me good 2,3 line promt for my llm app where you take questions as a Subject { subject } then question { Question } if the question is not related to the given subject say sorry the question is not releated to the given subject, create a promt for system and user')

In [ ]:
print(command.content)

In [189]:
study_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert on {subject}.  Answer questions accurately.  If a question is outside this subject, apologize and state you cannot answer."),
    ("user", "Subject: {subject}, Question: {question}")
])

In [191]:
study_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Respond to questions about {subject} only. If a question is off-topic, politely decline to answer."),
    ("user", "Subject: {subject}, Question: {question}")
])

In [182]:
study_chain = study_prompt_template | llm | StrOutputParser()

In [197]:
answer = study_chain.invoke({
    "subject":'Computer Science',
    "question":"Explain the working of human heart"
})
print(answer)


I'm designed to answer questions about Computer Science. Explaining the workings of the human heart falls outside of that domain.


In [23]:
from langchain_core.messages import HumanMessage

In [32]:
with open('image.png','rb') as image_file:
    image_content = image_file.read()
    encoded_image = base64.b64encode(image_content).decode('utf-8')
print(encoded_image)


...


In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "can you tell me how i can send this local encoded image to gemini Api through langchain",
        },
        {"type": "image_url", "image_url": {"url":f"data:image/png;base64,{encoded_image}"}},
    ]
)

otp = llm.invoke([message])
print(otp.content)

Okay, here's how you can send the locally encoded image to the Gemini API using Langchain, based on the code you've provided. I'll outline the necessary steps and provide example code.

**Conceptual Outline**

1.  **Read and Encode the Image:**  (You've already done this!)  You're reading the image in binary mode, encoding it as base64, and decoding it to a UTF-8 string.  This is the correct approach.

2.  **Construct the Gemini API Request:**  You'll need to format the request as a list of `HumanMessage` objects, where one message contains the text prompt and the image data.

3.  **Use Langchain's Gemini API Integration:**  You'll use the `ChatGemini` class from Langchain to interact with the Gemini API.  You'll need your Gemini API key.

**Code Implementation**

```python
import base64
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# 1. Read and Encode the Image (as you've already done)
def encode_image(image_path):
    wit